In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
from pathlib import Path
import platform
import sys
import os
import json
import uuid
import socket
import requests
import webbrowser
from typing import Dict, List, Optional, Any

In [48]:
from bruker.api.topspin import Topspin
from bruker.data.nmr import *

In [56]:
import simpleNMRbrukerTools
print(simpleNMRbrukerTools.__version__)

# Import submodules
from simpleNMRbrukerTools import core
from simpleNMRbrukerTools import gui
from simpleNMRbrukerTools import parsers
from simpleNMRbrukerTools import utils

from simpleNMRbrukerTools.core.json_converter import BrukerToJSONConverter
from simpleNMRbrukerTools.core.data_reader import BrukerDataDirectory  
from simpleNMRbrukerTools.config import EXPERIMENT_CONFIGS

1.0.0


In [57]:
# GUIDATA imports
try:
    import guidata
    import guidata.dataset as gds
    import guidata.dataset.dataitems as gdi
    from guidata.dataset.datatypes import DataSet
    from guidata.dataset.dataitems import DirectoryItem
    GUIDATA_AVAILABLE = True
except ImportError:
    print("Error: GUIDATA not available. Please install guidata:")
    print("pip install guidata")
    sys.exit(1)

if GUIDATA_AVAILABLE:
    # import local guidataWarningDialogs
    from simpleNMRbrukerTools.gui.guidataWarningDialogs import WarningDialog, myGUIDATAwarn

In [51]:
def get_bruker_root_folder_from_identifier(path):
    """
    Check if 'pdata' is in the path, and return the path up to the parent of pdata's parent-1.
    """
    path = Path(path)
    parts = path.parts
    
    if 'pdata' in parts:
        pdata_index = parts.index('pdata')
        # Go back one more level (skip the parent of pdata too)
        return Path(*parts[:pdata_index-1])
    
    return None

(bruker.data.nmr.NMRDataSet,
 bruker.api.topspin.DataProvider,
 bruker.api.topspin.Topspin)

1.0.0


In [29]:
class BrukerFolderDialog(DataSet):
    """Dialog for selecting Bruker experiment folder."""
    bruker_folder = DirectoryItem("Bruker Data Folder", default=".")


def create_processing_class(expt_pdata_with_peaks, converter):
    class Processing(gds.DataSet):
        """Example"""
        
        expts = {}
        
        for expt_id, proc_files in expt_pdata_with_peaks.items():
            expt_data = converter.bruker_data[expt_id]
            experiment_type = expt_data.get('experimentType', 'Unknown')
            if experiment_type == "Unknown":
                continue
            
            procnumbers = [proc_file.name for proc_file in proc_files]
            procnumbers.append("SKIP")
            print(expt_id, procnumbers)
            
            expts[f"expt_{expt_id}"] = (gdi.ChoiceItem(f"{expt_id} {experiment_type}", procnumbers))
            locals()[f"expt_{expt_id}"] = expts[f"expt_{expt_id}"]

        simulated_annealing = gdi.BoolItem("Use simulated annealing", default=True)
        ml_consent = gdi.BoolItem("Permit Data to be saved to build Database", default=False)
    
    return Processing

def create_processing_dialog(experiments_with_peaks: Dict[str, List], converter):
    """
    Dynamically create a processing dialog based on available experiments.
    
    Args:
        experiments_with_peaks: Dictionary mapping experiment IDs to available processing folders
        converter: BrukerToJSONConverter instance
        
    Returns:
        DataSet class for the processing dialog
    """
    class Processing(gds.DataSet):
        """Example"""
        
        _experiment_choices = {}
        
        for expt_id, proc_files in experiments_with_peaks.items():
            expt_data = converter.bruker_data[expt_id]
            experiment_type = expt_data.get('experimentType', 'Unknown')
            if experiment_type == "Unknown":
                continue
            
            procnumbers = [proc_file.name for proc_file in proc_files]
            procnumbers.append("SKIP")
            print(expt_id, procnumbers)

            _experiment_choices[f"expt_{expt_id}"] = (gdi.ChoiceItem(f"{expt_id} {experiment_type}", procnumbers))
            locals()[f"expt_{expt_id}"] = _experiment_choices[f"expt_{expt_id}"]

        simulated_annealing = gdi.BoolItem("Use simulated annealing", 
                                           default=True,
                                           help="Enable simulated annealing of COSY and HMBC for structure optimization")
        ml_consent = gdi.BoolItem("Permit Data to be saved to build Database", 
                                  default=False,
                                  help="Allow your data to contribute to improving NMR prediction models")
    
    return Processing
 


def check_user_registration() -> bool:
    """
    Check if the user's machine is registered for the service.
    
    Returns:
        True if user can proceed, False otherwise
    """
    try:
        # Generate machine ID (MAC address based)
        mac_based_id = hex(uuid.getnode())
        print(f"Machine ID: {mac_based_id}")
        
        # Prepare request
        json_obj = {"hostname": mac_based_id}
        entry_point = "https://test-simplenmr.pythonanywhere.com/check_machine_learning"
        
        print("Checking user registration...")
        
        # Make the POST request
        response = requests.post(
            entry_point,
            headers={'Content-Type': 'application/json'},
            json=json_obj,
            timeout=10
        )
        
        print(f"Registration check response: {response.status_code}")
        
        if response.status_code == 200:
            try:
                response_data = response.json()
            except json.JSONDecodeError:
                print("Invalid JSON response from server.")
                myGUIDATAwarn("Invalid JSON response from server.")
                return False
            
            status = response_data.get("status", False)
            
            if isinstance(status, str) and status.strip().lower() == "unregistered":
                print("Machine is unregistered. Opening registration page...")
                registration_url = response_data.get("registration_url", "")
                if registration_url:
                    webbrowser.open(registration_url)
                else:
                    print("No registration URL provided.")
                myGUIDATAwarn("No registration URL provided.")
                return False
            
            elif isinstance(status, str) and status.strip().lower() == "registered":
                print("Machine is registered. Proceeding...")
                return True
            
            elif isinstance(status, bool) and not status:
                print("Registration status unclear.")
                myGUIDATAwarn("Registration status unclear.")
                return False
            
        else:
            print(f"Registration check failed: {response.status_code} - {response.text}")
            myGUIDATAwarn(f"Registration check failed: {response.status_code} - {response.text}")
            
    except requests.RequestException as e:
        print(f"Network error during registration check: {e}")
        print("Proceeding without registration check...")
        myGUIDATAwarn("Network error during registration check. Proceeding without registration check.")
        return True  # Allow offline usage
    except Exception as e:
        print(f"Error during registration check: {e}")
        myGUIDATAwarn(f"Error during registration check: {e}")
        
    return False


def find_experiments_with_peaks(converter) -> Dict[str, List]:
    """
    Find experiments that have peak data available.
    
    Args:
        converter: BrukerToJSONConverter instance
        
    Returns:
        Dictionary mapping experiment IDs to lists of processing folders with peaks
    """
    experiments_with_peaks = {}
    
    # Handle both original and refactored data structures
    if hasattr(converter, 'bruker_data'):
        # Refactored structure
        data_dict = converter.bruker_data.data if hasattr(converter.bruker_data, 'data') else converter.bruker_data
    else:
        # Original structure
        data_dict = converter._all_bruker_folders
    
    for expt_id, expt_data in data_dict.items():
        if not expt_data.get('haspeaks', False):
            continue
            
        experiment_type = expt_data.get('experimentType', 'Unknown')
        if experiment_type == 'Unknown':
            continue
        
        # Find processing folders with peaks
        pdata = expt_data.get('pdata', {})
        proc_folders_with_peaks = []
        
        # Handle different pdata structures
        if 'procfolders' in pdata:
            # Refactored structure
            for folder in pdata.get('procfolders', []):
                folder_name = folder.name if hasattr(folder, 'name') else str(folder)
                proc_data = pdata.get(folder_name, {})
                
                if proc_data.get('haspeaks', False):
                    proc_folders_with_peaks.append(folder)
        else:
            # Original structure - check for numbered folders
            for key, value in pdata.items():
                if key != 'path' and isinstance(value, dict):
                    if value.get('haspeaks', False):
                        proc_folders_with_peaks.append(key)
        
        if proc_folders_with_peaks:
            experiments_with_peaks[expt_id] = proc_folders_with_peaks
            print(f"Found experiment {expt_id} ({experiment_type}) with {len(proc_folders_with_peaks)} processed datasets")
    
    return experiments_with_peaks


def process_user_selections(dialog_instance, experiments_with_peaks: Dict, converter) -> Dict[str, Dict]:
    """
    Process user selections from the dialog.
    
    Args:
        dialog_instance: Instance of the ProcessingDialog
        experiments_with_peaks: Available experiments
        converter: BrukerToJSONConverter instance
        
    Returns:
        Dictionary of user selections for conversion
    """
    user_selections = {}
    
    # Handle both data structures
    if hasattr(converter, 'bruker_data'):
        data_dict = converter.bruker_data.data if hasattr(converter.bruker_data, 'data') else converter.bruker_data
    else:
        data_dict = converter._all_bruker_folders
    
    for expt_id in experiments_with_peaks.keys():
        expt_data = data_dict[expt_id]
        experiment_type = expt_data.get('experimentType', 'Unknown')
        
        if experiment_type == "Unknown":
            continue
        
        attr_name = f"expt_{expt_id}"
        if hasattr(dialog_instance, attr_name):
            selected_index = getattr(dialog_instance, attr_name)
            
            # Get the choice item to access the options
            choice_item = dialog_instance._experiment_choices[attr_name]
            choices = choice_item.get_prop("data", "choices")
            
            # Convert index to actual choice text
            if 0 <= selected_index < len(choices):
                selected_choice = choices[selected_index][1]  # choices is list of (value, label) tuples
                
                print(f"User selected: {expt_id} ({experiment_type}) -> {selected_choice}")
                
                if selected_choice != "SKIP":
                    user_selections[expt_id] = {
                        "experimentType": experiment_type,
                        "procno": selected_choice
                    }
    
    return user_selections


def submit_to_server(json_data: Dict) -> bool:
    """
    Submit the JSON data to the processing server.
    
    Args:
        json_data: The converted JSON data
        
    Returns:
        True if successful, False otherwise
    """
    try:
        print("Submitting data to processing server...")
        
        response = requests.post(
            'https://test-simplenmr.pythonanywhere.com/simpleMNOVA',
            headers={'Content-Type': 'application/json'},
            json=json_data,
            timeout=30
        )
        
        print(f"Server response: {response.status_code}")
        
        if response.status_code == 200:
            # Save response to file
            with open('nmr_analysis_result.html', 'w', encoding='utf-8') as f:
                f.write(response.text)
            
            print("✓ Analysis complete! Results saved to 'nmr_analysis_result.html'")
            
            # Open in browser
            result_path = Path('nmr_analysis_result.html').absolute()
            webbrowser.open(f'file://{result_path}')
            
            return True
        else:
            print(f"Server error: {response.status_code} - {response.text}")
            return False
            
    except requests.RequestException as e:
        print(f"Network error: {e}")
        return False
    except Exception as e:
        print(f"Error submitting to server: {e}")
        return False


In [30]:
# Initialize QApplication for GUIDATA
_app = guidata.qapplication()

In [58]:
top = Topspin()
dp = top.getDataProvider()
cdataset = dp.getCurrentDataset()
type(cdataset), type(dp), type(top)

(bruker.data.nmr.NMRDataSet,
 bruker.api.topspin.DataProvider,
 bruker.api.topspin.Topspin)

In [59]:
brukerRootFolder = Path()
if isinstance(cdataset, type(None)):
    print("Please load a data set that you are working on into Topspin")
else:
    brukerRootFolder = get_bruker_root_folder_from_identifier(cdataset.getIdentifier())
    bruker_expt_folder = get_bruker_root_folder_from_identifier(cdataset.getIdentifier())
    
brukerRootFolder

PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1')

In [60]:
if isinstance(brukerRootFolder, type(None)):
    print("No valid Bruker root folder found. Loading default folder for now.")
    if platform.system() == "Windows":
        bruker_expt_folder = Path(r"C:\Users\vsmw51\OneDrive - Durham University\projects\programming\2025\python\awh\bruker_data_sets\E72507_04164043_propyl_benzoate\04164043")
    if platform.system() == "Darwin":
        bruker_expt_folder = Path("/Users/vsmw51/OneDrive - Durham University/projects/programming/2025/python/awh/bruker_data_sets/E72507_04164043_propyl_benzoate/04164043")

In [61]:
converter = BrukerToJSONConverter(bruker_expt_folder)

Experiment 1 identified as H1_1D
Experiment 4 identified as HMBC
Experiment 3 identified as HSQC
Experiment 2 identified as COSY
Experiment 5 identified as C13_1D
Found 1 mol file(s): ['alpha-ionon.mol']
Selected mol file: alpha-ionon.mol
Successfully loaded mol file: alpha-ionon.mol
Molecule has 14 atoms
Generated SMILES from mol file: CC(=O)/C=C/C1C(C)=CCCC1(C)C


In [62]:
if not check_user_registration():
    print("\n❌ Unable to verify registration. Please check your internet connection")
    print("   or contact support at simpleNMR@gmail.com for assistance.")
    input("Press Enter to exit...")
    myGUIDATAwarn("Unable to verify registration. Please check your internet connection or contact support.")
else:
    myGUIDATAwarn("Registration check passed. Proceeding with data processing...")
    

Machine ID: 0x624bb2cbbfd0
Checking user registration...
Network error during registration check: HTTPSConnectionPool(host='test-simplenmr.pythonanywhere.com', port=443): Read timed out. (read timeout=10)
Proceeding without registration check...


In [70]:
# Step 1: Select Bruker data folder
folder_dialog = BrukerFolderDialog(title="Select Bruker Data Folder")
# set default folder
folder_dialog.bruker_folder = str(brukerRootFolder)

In [73]:
if not folder_dialog.edit():
    print("No folder selected. Exiting.")
    myGUIDATAwarn("No folder selected. Exiting.")
    # return

No folder selected. Exiting.


In [37]:
bruker_data_dir = Path(folder_dialog.bruker_folder)
print(f"Selected folder: {bruker_data_dir}")

if not bruker_data_dir.exists():
    # print(f"❌ Error: Directory does not exist: {bruker_data_dir}")
    myGUIDATAwarn(f"Error: Directory does not exist: {bruker_data_dir}")
    # return

Selected folder: /opt/topspin4.5.0/examdata/exam_CMCse_1


In [74]:
# Load and analyze Bruker data
print("\n2. Analyzing Bruker Data...")
try:
    converter = BrukerToJSONConverter(bruker_data_dir)
    # Handle both data structures
    if hasattr(converter, 'bruker_data'):
        data_count = len(converter.bruker_data.data if hasattr(converter.bruker_data, 'data') else converter.bruker_data)
    else:
        data_count = len(converter._all_bruker_folders)
    print(f"✓ Found {data_count} experiment folders")
    myGUIDATAwarn(f"✓ Found {data_count} experiment folders")
except Exception as e:
    print(f"❌ Error loading Bruker data: {e}")
    myGUIDATAwarn(f"Error loading Bruker data: {e}")
    import traceback
    traceback.print_exc()
    # return


2. Analyzing Bruker Data...
Experiment 1 identified as H1_1D
Experiment 4 identified as HMBC
Experiment 3 identified as HSQC
Experiment 2 identified as COSY
Experiment 5 identified as C13_1D
Found 1 mol file(s): ['alpha-ionon.mol']
Selected mol file: alpha-ionon.mol
Successfully loaded mol file: alpha-ionon.mol
Molecule has 14 atoms
Generated SMILES from mol file: CC(=O)/C=C/C1C(C)=CCCC1(C)C
✓ Found 5 experiment folders


In [75]:
# Step 3: Find experiments with peaks
print("\n3. Finding Experiments with Peak Data...")
experiments_with_peaks = find_experiments_with_peaks(converter)

if not experiments_with_peaks:
    print("❌ No experiments with peak data found.")
    print("   Make sure your data contains processed spectra with peak lists.")
    myGUIDATAwarn("No experiments with peak data found.\n   Make sure your data contains processed spectra with peak lists.")
    # return

print(f"✓ Found {len(experiments_with_peaks)} experiments with peak data")
myGUIDATAwarn(f"✓ Found {len(experiments_with_peaks)} experiments with peak data")


3. Finding Experiments with Peak Data...
Found experiment 3 (HSQC) with 1 processed datasets
Found experiment 5 (C13_1D) with 1 processed datasets
✓ Found 2 experiments with peak data


1024

In [77]:
experiments_with_peaks.values()

dict_values([[PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/3/pdata/1')], [PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata/1')]])

In [83]:
# check if HSQC is one of the experiments with peaks
hsqc_with_peaks = False
for expt_id, proc_folders in experiments_with_peaks.items():
    # find id in coverter
    expt = converter.bruker_data[expt_id]
    if expt["experimentType"] == "HSQC":
        print(f"Found HSQC experiment in  {expt_id} with {len(proc_folders)} processing folders")
        hsqc_with_peaks = True
        break

Found HSQC experiment in  3 with 1 processing folders


In [80]:
expt

{'path': PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5'),
 'dimensions': 1,
 'acqu_files': [PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/acqu'),
  PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/acqus')],
 'acqu': <simpleNMRbrukerTools.parsers.parameter_parser.BrukerParameterFile at 0x1898aac30>,
 'acqus': <simpleNMRbrukerTools.parsers.parameter_parser.BrukerParameterFile at 0x1898aaed0>,
 'pulseprogram': 'zgpg30',
 'nuclei': ['13C'],
 'pdata': {'path': PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata'),
  'procfolders': [PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata/1')],
  '1': {'path': PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata/1'),
   'proc_files': [PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata/1/proc'),
    PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata/1/procs')],
   'proc': <simpleNMRbrukerTools.parsers.parameter_parser.BrukerParameterFile at 0x1898aaf90>,
   'procs': <simpleNMRbrukerTools.parser

In [40]:
experiments_with_peaks

{'3': [PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/3/pdata/1')],
 '5': [PosixPath('/opt/topspin4.5.0/examdata/exam_CMCse_1/5/pdata/1')]}

In [41]:
# Step 4: Create and show processing dialog
print("\n4. Experiment Selection Dialog")
ProcessingDialog = create_processing_dialog(experiments_with_peaks, converter)
dialog_instance = ProcessingDialog()

if not dialog_instance.edit():
    print("Dialog cancelled. Exiting.")
    # return


4. Experiment Selection Dialog
3 ['1', 'SKIP']
5 ['1', 'SKIP']


In [42]:
# Step 5: Process user selections
print("\n5. Processing User Selections...")
user_selections = process_user_selections(dialog_instance, experiments_with_peaks, converter)

if not user_selections:
    print("❌ No experiments selected for processing.")
    myGUIDATAwarn("No experiments selected for processing.")
    # return

else:

    print(f"✓ Selected {len(user_selections)} experiments for processing")
    myGUIDATAwarn(f"✓ Selected {len(user_selections)} experiments for processing")

    # Get processing options
    ml_consent = dialog_instance.ml_consent
    simulated_annealing = dialog_instance.simulated_annealing

    print(f"  - ML consent: {ml_consent}")
    print(f"  - Simulated annealing: {simulated_annealing}")


5. Processing User Selections...
User selected: 3 (HSQC) -> 1
User selected: 5 (C13_1D) -> 1
✓ Selected 2 experiments for processing
  - ML consent: False
  - Simulated annealing: True


In [43]:
# Step 6: Convert to JSON
print("\n6. Converting to JSON...")
try:
    json_data = converter.convert_to_json(
        user_expt_selections=user_selections,
        ml_consent=ml_consent,
        simulated_annealing=simulated_annealing
    )
    print("type(json_data):", type(json_data))
    print("✓ JSON conversion complete")
    myGUIDATAwarn("✓ JSON conversion complete")
except Exception as e:
    print(f"❌ Error during JSON conversion: {e}")
    myGUIDATAwarn(f"Error during JSON conversion: {e}")
    import traceback
    traceback.print_exc()
    # return


6. Converting to JSON...
Created atom info from mol file: 14 total atoms, 13 carbon atoms
Processing experiment 3 with type HSQC
Processing experiment 5 with type C13_1D
type(json_data): <class 'dict'>
✓ JSON conversion complete


In [44]:
# Step 7: Save JSON file locally
output_filename = converter.data_directory / f"{converter.data_directory.name}_assignments.json"
try:
    converter.save_json(output_filename)
    print(f"✓ JSON file saved: {output_filename}")
    myGUIDATAwarn(f"✓ JSON file saved: {output_filename}")
except Exception as e:
    print(f"⚠️  Warning: Could not save JSON file: {e}")
    myGUIDATAwarn(f"Warning: Could not save JSON file: {e}")

JSON data saved to: /opt/topspin4.5.0/examdata/exam_CMCse_1/exam_CMCse_1_assignments.json
✓ JSON file saved: /opt/topspin4.5.0/examdata/exam_CMCse_1/exam_CMCse_1_assignments.json


In [45]:
type(converter.data_directory)

pathlib._local.PosixPath

In [46]:
# Step 8: Submit to server for analysis
print("\n7. Submitting to Analysis Server...")
if submit_to_server(json_data):
    print("✓ Analysis complete! Check the opened browser window for results.")
    myGUIDATAwarn("✓ Analysis complete! Check the opened browser window for results.")
else:
    print("⚠️  Server submission failed, but JSON file was saved locally.")
    myGUIDATAwarn("Server submission failed, but JSON file was saved locally.")

print("\n" + "=" * 60)
print("Processing Complete!")
print("=" * 60)


7. Submitting to Analysis Server...
Submitting data to processing server...
Server response: 200
✓ Analysis complete! Results saved to 'nmr_analysis_result.html'
✓ Analysis complete! Check the opened browser window for results.

Processing Complete!


In [ ]:
json_data["workingDirectory"]["data"]["0"]

'c:/Users/vsmw51/OneDrive - Durham University/projects/programming/2025/python/awh/simpleNMRbrukerTools'

In [ ]:
json_data["workingFilename"]["data"]["0"]

''

In [ ]:
json_data.keys()

dict_keys(['hostname', 'workingDirectory', 'workingFilename', 'allAtomsInfo', 'carbonAtomsInfo', 'nmrAssignments', 'c13predictions', 'chosenSpectra', 'exptIdentifiers', 'spectraWithPeaks', 'carbonCalcPositionsMethod', 'MNOVAcalcMethod', 'randomizeStart', 'startingTemperature', 'endingTemperature', 'coolingRate', 'numberOfSteps', 'ppmGroupSeparation', 'ml_consent', 'simulatedAnnealing'])

In [ ]:
type(json_data)

dict

In [ ]:
dir(converter)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_atom_info',
 '_add_experiment_settings',
 '_add_ml_consent',
 '_add_molecular_info',
 '_add_nmr_spectra',
 '_add_processing_parameters',
 '_add_simulated_annealing',
 '_add_system_info',
 '_convert_2d_integrals_to_json',
 '_convert_peaklist_to_json',
 '_create_all_atoms_info_from_mol',
 '_create_carbon_atoms_info_from_mol',
 '_create_spectrum_entry',
 '_get_experiment_type_string',
 '_get_integrals_data',
 '_get_peaks_data',
 '_get_probe_info',
 '_get_spec_frequency',
 '_get_spectrum_subtype',
 '_get_temperature',
 '_process_mol_files',
 'bruker_data',
 'convert_to_json',
 'data_directory',
 'find_m

In [ ]:
from dataclasses import dataclass
from typing import Optional, Union

@dataclass
class Result:
    success: bool
    value: Optional[any] = None
    error: Optional[str] = None

def process_data(data):
    try:
        if not data:
            return Result(success=False, error="Empty data provided")
        
        processed = data # function goes here, e.g., expensive_operation(data)
        return Result(success=True, value=processed)
    
    except Exception as e:
        return Result(success=False, error=str(e))

# Usage
my_data = None
result = process_data(my_data)
if result.success:
    print("Success:", result.value)
else:
    print("Failed:", result.error)

Failed: Empty data provided


In [ ]:
dir(result)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'error',
 'success',
 'value']

In [ ]:
result.error

'Empty data provided'

In [ ]:
result

Result(success=False, value=None, error='Empty data provided')

In [ ]:
print(result)

Result(success=False, value=None, error='Empty data provided')


In [ ]:
result.__repr__()

"Result(success=False, value=None, error='Empty data provided')"